In [1]:
# Imports and setup
from pathlib import Path
from typing import List
from langchain.document_loaders import TextLoader, CSVLoader, JSONLoader, UnstructuredMarkdownLoader
from langchain.schema import Document

# Path to enterprise documents
KNOWLEDGE_BASE_PATH = Path("enterprise_knowledge_base")

print(" Setup complete!")

Setup complete!


In [2]:
def load_enterprise_documents(base_path: Path) -> List[Document]:
    """Load all documents recursively with proper metadata"""
    
    all_docs = []
    
    print(" Loading DataFlow's documents...")
    
    # Process each department folder
    for dept_path in base_path.iterdir():
        if not dept_path.is_dir():
            continue
            
        department = dept_path.name
        print(f" {department}...")
        
        # Get ALL files recursively
        files = [f for f in dept_path.rglob("*") if f.is_file()]
        
        for file_path in files:
            try:
                # Choose loader by extension
                ext = file_path.suffix.lower()
                if ext == '.csv':
                    loader = CSVLoader(str(file_path))
                elif ext == '.json':
                    loader = JSONLoader(str(file_path), jq_schema='.', text_content=False)
                elif ext == '.md':
                    loader = UnstructuredMarkdownLoader(str(file_path))
                else:
                    loader = TextLoader(str(file_path), encoding='utf-8')
                
                # Load and add metadata
                docs = loader.load()
                for doc in docs:
                    doc.metadata.update({
                        "department": department,
                        "source_file": file_path.name,
                        "file_type": ext
                    })
                
                all_docs.extend(docs)
                rel_path = file_path.relative_to(dept_path)
                print(f"    {rel_path}")
                
            except Exception as e:
                print(f"    {file_path.name}: {str(e)[:30]}...")
    
    # Quick summary
    departments = set(doc.metadata['department'] for doc in all_docs)
    total_chars = sum(len(doc.page_content) for doc in all_docs)
    
    print(f"\n LOADED: {len(all_docs)} documents from {len(departments)} departments")
    print(f" Content: {total_chars:,} characters")
    print(f" Departments: {', '.join(sorted(departments))}")
    
    return all_docs

# Load all documents
documents = load_enterprise_documents(KNOWLEDGE_BASE_PATH)

Loading DataFlow's documents...
 business_data...
    billing_and_pricing.csv
    customer_analytics.csv
   integration_partners.csv
 customer_facing...
    api_documentation.json
    competitive_analysis.txt
    product_user_guide.markdown
    terms_of_service.markdown
    troubleshooting_guide.txt
 internal_operations...
    hr_policies\employee_handbook.txt
    hr_policies\onboarding_checklist.json
    product_releases\release_notes.json
    sales_marketing\sales_playbook.json
    support_operations\customer_support_procedures.markdown
    support_operations\system_architecture.markdown
 legal_compliance...
    compliance_certifications.csv
    privacy_policy.txt
    security_policies.txt
    terms_of_service.markdown

 LOADED: 212 documents from 4 departments
 Content: 262,608 characters
 Departments: business_data, customer_facing, internal_operations, legal_compliance


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
 
def create_smart_chunks(documents: List[Document]) -> List[Document]:
    """Split documents into optimal chunks for RAG"""
    
    print(" Creating smart chunks...")
    
    # Industry-standard chunking settings
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,        # Optimal for embedding models
        chunk_overlap=200,      # Preserve context
        length_function=len,    # Character-based
        separators=[            # Try these in order:
            "\n\n",              # Paragraphs first
            "\n",                # Then lines
            ". ",                # Then sentences
            " ",                 # Then words
            "",                  # Finally characters
        ]
    )
    
    all_chunks = []
    stats = {
        "original_docs": len(documents),
        "total_chunks": 0,
        "by_department": {}
    }
    
    # Process each department
    for dept in set(doc.metadata['department'] for doc in documents):
        dept_docs = [doc for doc in documents if doc.metadata['department'] == dept]
        dept_chunks = []
        
        print(f" {dept}: {len(dept_docs)} docs → ", end="")
        
        for doc in dept_docs:
            # Split the document
            chunks = text_splitter.split_documents([doc])
            
            # Add chunk metadata
            for i, chunk in enumerate(chunks):
                chunk.metadata.update({
                    "chunk_id": f"{doc.metadata['source_file']}_{i}",
                    "chunk_index": i,
                    "total_chunks": len(chunks),
                    "chunk_size": len(chunk.page_content)
                })
            
            dept_chunks.extend(chunks)
        
        stats["by_department"][dept] = len(dept_chunks)
        stats["total_chunks"] += len(dept_chunks)
        all_chunks.extend(dept_chunks)
        
        print(f"{len(dept_chunks)} chunks")
    
    print(f"\n CHUNKING COMPLETE:")
    print(f"    Original: {stats['original_docs']} documents")
    print(f"    Created: {stats['total_chunks']} chunks")
    print(f"    Ratio: {stats['total_chunks'] / stats['original_docs']:.1f} chunks per document")
    
    return all_chunks
 
# Create chunks
chunks = create_smart_chunks(documents)

 Creating smart chunks...
 business_data: 173 docs → 173 chunks
 legal_compliance: 28 docs → 80 chunks
 customer_facing: 5 docs → 105 chunks
 internal_operations: 6 docs → 119 chunks

 CHUNKING COMPLETE:
    Original: 212 documents
    Created: 477 chunks
    Ratio: 2.2 chunks per document


In [5]:
def analyze_chunk_quality(chunks: List[Document]):
    """Analyze chunk distribution and quality"""
    
    print(" CHUNK QUALITY ANALYSIS")
    print("-" * 30)
    
    # Size analysis
    sizes = [len(chunk.page_content) for chunk in chunks]
    avg_size = sum(sizes) / len(sizes)
    min_size = min(sizes)
    max_size = max(sizes)
    
    print(f" Size Distribution:")
    print(f"   Average: {avg_size:.0f} characters")
    print(f"   Range: {min_size} - {max_size} characters")
    
    # Size buckets
    buckets = {
        "Small (0-500)": sum(1 for s in sizes if s <= 500),
        "Medium (500-1000)": sum(1 for s in sizes if 500 < s <= 1000),
        "Large (1000+)": sum(1 for s in sizes if s > 1000)
    }
    
    print(f"\n Size Distribution:")
    for bucket, count in buckets.items():
        percentage = (count / len(chunks)) * 100
        print(f"   {bucket}: {count} chunks ({percentage:.1f}%)")
    
    # Department distribution
    by_dept = {}
    for chunk in chunks:
        dept = chunk.metadata['department']
        by_dept[dept] = by_dept.get(dept, 0) + 1
    
    print(f"\n By Department:")
    for dept, count in sorted(by_dept.items()):
        percentage = (count / len(chunks)) * 100
        print(f"   {dept}: {count} chunks ({percentage:.1f}%)")
    
    # Quality assessment
    optimal_chunks = sum(1 for s in sizes if 500 <= s <= 1000)
    quality_score = (optimal_chunks / len(chunks)) * 100
    
    print(f"\n Quality Score: {quality_score:.1f}%")
    print(f"   ({optimal_chunks}/{len(chunks)} chunks in optimal range)")
    
    if quality_score >= 70:
        print(" Excellent chunking quality!")
    elif quality_score >= 50:
        print(" Good chunking quality")
    else:
        print(" Consider adjusting chunk size")
 
analyze_chunk_quality(chunks)

 CHUNK QUALITY ANALYSIS
------------------------------
 Size Distribution:
   Average: 586 characters
   Range: 3 - 1000 characters

 Size Distribution:
   Small (0-500): 222 chunks (46.5%)
   Medium (500-1000): 255 chunks (53.5%)
   Large (1000+): 0 chunks (0.0%)

 By Department:
   business_data: 173 chunks (36.3%)
   customer_facing: 105 chunks (22.0%)
   internal_operations: 119 chunks (24.9%)
   legal_compliance: 80 chunks (16.8%)

 Quality Score: 53.5%
   (255/477 chunks in optimal range)
 Good chunking quality


In [6]:
# Modern imports (no deprecation warnings)
try:
    from langchain_huggingface import HuggingFaceEmbeddings
    print(" Using modern langchain-huggingface (recommended)")
    modern_import = True
except ImportError:
    from langchain.embeddings import HuggingFaceEmbeddings
    print(" Using deprecated import (consider upgrading)")
    modern_import = False
 
from langchain.vectorstores import FAISS
import numpy as np
 
def setup_embedding_model():
    """Initialize the embedding model for vector creation"""
    
    print(" Loading embedding model...")
    
    # Use production-grade embedding model
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    
    # Modern LangChain wrapper (no deprecation warnings)
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={'device': 'cpu'},  # Use CPU for compatibility
        encode_kwargs={'normalize_embeddings': True}  # Better for similarity search
    )
    
    print(f" Model loaded: {model_name}")
    print(f" Vector dimensions: 384")
    print(f" Device: CPU (production compatible)")
    
    if modern_import:
        print(" Using modern non-deprecated embeddings!")
    
    return embeddings
 
# Setup embeddings
embeddings = setup_embedding_model()

 Using modern langchain-huggingface (recommended)
 Loading embedding model...


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Michael\OneDrive\Desktop\Enterprise_RAG_customer_serviceAgent\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Michael\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Model loaded: sentence-transformers/all-MiniLM-L6-v2
 Vector dimensions: 384
 Device: CPU (production compatible)
 Using modern non-deprecated embeddings!


In [7]:
def create_vector_store(chunks: List, embeddings) -> FAISS:
    """Create FAISS vector store from text chunks"""
    
    print(" Creating vector embeddings...")
    print(" This may take 30-60 seconds...")
    
    # Create vector store with FAISS
    vector_store = FAISS.from_documents(
        documents=chunks,
        embedding=embeddings
    )
    
    print(f" Vector store created!")
    print(f" Vectors: {len(chunks)}")
    print(f" Dimensions: 384 per vector")
    print(f" Total size: ~{len(chunks) * 384 * 4 / 1024 / 1024:.1f} MB")
    
    return vector_store
 
# Create the vector store
vector_store = create_vector_store(chunks, embeddings)
 
# Test semantic search
def test_semantic_search(vector_store: FAISS):
    """Test the vector store with realistic customer queries"""
    
    print("\n TESTING SEMANTIC SEARCH")
    print("-" * 30)
    
    test_queries = [
        "What are your pricing plans?",
        "How do I integrate with your API?", 
        "What is your privacy policy?",
        "I'm having trouble with authentication"
    ]
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n Query {i}: '{query}'")
        
        # Search for most relevant chunks
        results = vector_store.similarity_search(query, k=3)
        
        print(f" Found {len(results)} relevant chunks:")
        
        for j, result in enumerate(results, 1):
            dept = result.metadata['department']
            file = result.metadata['source_file']
            preview = result.page_content[:80].replace('\n', ' ')
            
            print(f"   {j}.  {dept} | 📄 {file}")
            print(f"      Preview: {preview}...")
 
test_semantic_search(vector_store)

 Creating vector embeddings...
 This may take 30-60 seconds...
 Vector store created!
 Vectors: 477
 Dimensions: 384 per vector
 Total size: ~0.7 MB

 TESTING SEMANTIC SEARCH
------------------------------

 Query 1: 'What are your pricing plans?'
 Found 3 relevant chunks:
   1.  business_data | 📄 billing_and_pricing.csv
      Preview: Plan_Type: Pricing Feature: Base price (USD Starter_Plan: annual) Professional_P...
   2.  business_data | 📄 billing_and_pricing.csv
      Preview: Plan_Type: Add-Ons Feature: Priority data processing Starter_Plan: No Profession...
   3.  business_data | 📄 billing_and_pricing.csv
      Preview: Plan_Type: Add-Ons Feature: Dedicated compute Starter_Plan: No Professional_Plan...

 Query 2: 'How do I integrate with your API?'
 Found 3 relevant chunks:
   1.  customer_facing | 📄 api_documentation.json
      Preview: . Includes methods for dashboards and data sources."}, {"language": "JavaScript"...
   2.  customer_facing | 📄 api_documentation.json
      Prev

In [8]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import Ollama
 
# Setup local LLM (free, no API costs)
try:
    llm = Ollama(model="llama3.2", base_url="http://localhost:11434")
    test_response = llm.invoke("Hello")
    print(" Ollama LLM connected successfully!")
    print(" Using free local LLM")
except Exception as e:
    print(f" Ollama connection failed: {e}")
    print(" Make sure Ollama is running: ollama serve")
    llm = None
 
# Professional customer service prompt
CUSTOMER_SERVICE_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are DataFlow's helpful customer service assistant. Your job is to provide accurate, friendly, and professional support to customers.
 
INSTRUCTIONS:
- Use the provided context to answer questions accurately
- Be concise but thorough in your explanations
- If information isn't in the context, say "I don't have that specific information" and suggest contacting support
- Always maintain a helpful and professional tone
- For technical questions, provide step-by-step guidance when possible
 
CONTEXT:
{context}
 
CUSTOMER QUESTION:
{question}
 
RESPONSE:"""
)
 
def create_rag_chain(vector_store, llm, prompt_template):
    """Create production RAG chain"""
    
    if not llm:
        print(" No LLM available - cannot create RAG chain")
        return None
    
    print(" Creating RAG chain...")
    
    # Create retrieval QA chain
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Stuff all context into prompt
        retriever=vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}  # Retrieve top 4 most relevant chunks
        ),
        chain_type_kwargs={
            "prompt": prompt_template
        },
        return_source_documents=True  # Show which documents were used
    )
    
    print(" RAG chain created successfully!")
    print(" Retriever: Top 4 most relevant chunks")
    print(" LLM: Ready for customer questions")
    print(" Source attribution: Enabled")
    
    return rag_chain
 
# Create the RAG chain
rag_chain = create_rag_chain(vector_store, llm, CUSTOMER_SERVICE_PROMPT)

C:\Users\Michael\AppData\Local\Temp\ipykernel_19360\1781630592.py:7: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2", base_url="http://localhost:11434")


 Ollama LLM connected successfully!
 Using free local LLM
 Creating RAG chain...
 RAG chain created successfully!
 Retriever: Top 4 most relevant chunks
 LLM: Ready for customer questions
 Source attribution: Enabled


In [9]:
import time
from typing import Dict, List, Any
 
class DataFlowCustomerAgent:
    """Professional customer service agent with conversation tracking"""
    
    def __init__(self, rag_chain):
        self.rag_chain = rag_chain
        self.conversation_history = []
        self.conversation_count = 0
        self.response_times = []
        
        print(" DataFlow Customer Service Agent initialized")
    
    def ask(self, question: str) -> Dict[str, Any]:
        """Ask the agent a question and get a comprehensive response"""
        
        if not self.rag_chain:
            return {
                "answer": "I'm sorry, but I'm not properly configured right now. Please contact our support team directly.",
                "sources": [],
                "response_time": 0,
                "error": "No LLM available"
            }
        
        start_time = time.time()
        
        try:
            # Get response from RAG chain
            response = self.rag_chain.invoke({"query": question})
            
            end_time = time.time()
            response_time = end_time - start_time
            
            # Track conversation
            self.conversation_history.append({
                "question": question,
                "answer": response["result"],
                "timestamp": start_time
            })
            
            # Track metrics
            self.conversation_count += 1
            self.response_times.append(response_time)
            
            # Extract source information
            sources = []
            if "source_documents" in response:
                for doc in response["source_documents"]:
                    sources.append({
                        "department": doc.metadata.get("department", "unknown"),
                        "file": doc.metadata.get("source_file", "unknown"),
                        "preview": doc.page_content[:100] + "..."
                    })
            
            return {
                "answer": response["result"],
                "sources": sources,
                "response_time": response_time,
                "conversation_turn": self.conversation_count
            }
            
        except Exception as e:
            return {
                "answer": f"I apologize, but I encountered an error. Please try rephrasing or contact support.",
                "sources": [],
                "response_time": time.time() - start_time,
                "error": str(e)
            }
    
    def get_stats(self) -> Dict[str, Any]:
        """Get agent performance statistics"""
        
        if not self.response_times:
            return {"conversations": 0, "avg_response_time": 0}
        
        return {
            "conversations": self.conversation_count,
            "avg_response_time": sum(self.response_times) / len(self.response_times),
            "fastest_response": min(self.response_times),
            "slowest_response": max(self.response_times)
        }
 
# Create the customer service agent
agent = DataFlowCustomerAgent(rag_chain)
print(" Customer service agent ready!")

 DataFlow Customer Service Agent initialized
 Customer service agent ready!


In [10]:
def test_customer_scenarios(agent):
    """Test agent with realistic customer service scenarios"""
    
    print(" TESTING CUSTOMER SERVICE SCENARIOS")
    print("=" * 45)
    
    # Realistic customer questions
    scenarios = [
        {
            "question": "What are your pricing plans and how much does the premium plan cost?",
            "category": "Billing",
            "expected_dept": "business_data"
        },
        {
            "question": "How do I authenticate with your API? I'm getting authentication errors.",
            "category": "Technical Support",
            "expected_dept": "customer_facing"
        },
        {
            "question": "What data do you collect and how do you protect my privacy?",
            "category": "Privacy/Legal",
            "expected_dept": "legal_compliance"
        }
    ]
    
    results = []
    
    for i, scenario in enumerate(scenarios, 1):
        print(f"\n Scenario {i}: {scenario['category']}")
        print(f"Question: {scenario['question']}")
        print("-" * 50)
        
        # Get agent response
        response = agent.ask(scenario["question"])
        
        print(f"Agent Response:")
        print(f"   {response['answer']}")  # Show complete response
        
        print(f"\n Sources Used:")
        for j, source in enumerate(response['sources'][:3], 1):  # Show top 3 sources
            print(f"   {j}.  {source['department']} - {source['file']}")
        
        print(f"\n Response Time: {response['response_time']:.2f} seconds")
        
        # Check if correct department was used
        dept_match = any(source['department'] == scenario['expected_dept'] for source in response['sources'])
        accuracy = " Accurate" if dept_match else " Needs Review"
        print(f" Department Accuracy: {accuracy}")
        
        results.append({
            "scenario": scenario,
            "response": response,
            "accurate": dept_match
        })
    
    return results

# Test the scenarios
test_results = test_customer_scenarios(agent)

 TESTING CUSTOMER SERVICE SCENARIOS

 Scenario 1: Billing
 Question: What are your pricing plans and how much does the premium plan cost?
--------------------------------------------------
 Agent Response:
   Hello! I'd be happy to help you with your question. Our pricing plans include:

1. **Professional Plan**: This is a monthly subscription that costs $500/month.
2. **Enterprise Plan**: This is a monthly subscription that costs $1000/month.

Additionally, we have an optional Premium Support add-on which includes priority escalation and dedicated Customer Success Management (CSM). The cost of the Premium Support add-on varies depending on your plan type:

* If you're on our Starter Plan, there is no additional cost for the Premium Support add-on.
* If you're on our Professional or Enterprise Plan, the Premium Support add-on costs an additional $200-$500/month, respectively.

Please let me know if you have any other questions or if there's anything else I can help you with!

 Sources 

In [11]:
def calculate_business_impact(agent, test_results):
    """Calculate measurable business impact and ROI"""
    
    print(" BUSINESS IMPACT ANALYSIS")
    print("=" * 30)
    
    # Get agent performance stats
    stats = agent.get_stats()
    
    # Calculate accuracy
    accurate_responses = sum(1 for result in test_results if result['accurate'])
    accuracy_rate = (accurate_responses / len(test_results)) * 100 if test_results else 0
    
    # Business metrics
    metrics = {
        "daily_customer_questions": 50,
        "avg_human_response_time": 300,  # 5 minutes
        "hourly_support_cost": 25,
        "working_days_per_year": 250,
        "ai_accuracy_rate": accuracy_rate,
        "ai_avg_response_time": stats.get('avg_response_time', 0)
    }
    
    # Calculate savings
    daily_human_hours = (metrics['daily_customer_questions'] * metrics['avg_human_response_time']) / 3600
    daily_ai_hours = (metrics['daily_customer_questions'] * metrics['ai_avg_response_time']) / 3600
    
    hours_saved_daily = daily_human_hours - daily_ai_hours
    daily_cost_savings = hours_saved_daily * metrics['hourly_support_cost']
    annual_savings = daily_cost_savings * metrics['working_days_per_year']
    
    print(f" PERFORMANCE METRICS:")
    print(f"   Accuracy Rate: {accuracy_rate:.1f}%")
    print(f"   Avg Response Time: {metrics['ai_avg_response_time']:.2f} seconds")
    print(f"   Questions Handled: {stats.get('conversations', 0)}")
    
    print(f"\n COST ANALYSIS:")
    print(f"   Human Response Time: {metrics['avg_human_response_time']} seconds avg")
    print(f"   AI Response Time: {metrics['ai_avg_response_time']:.1f} seconds avg")
    print(f"   Speed Improvement: {(metrics['avg_human_response_time']/metrics['ai_avg_response_time']):.1f}x faster")
    
    print(f"\n BUSINESS IMPACT:")
    print(f"   Hours Saved Daily: {hours_saved_daily:.1f} hours")
    print(f"   Daily Cost Savings: ${daily_cost_savings:.2f}")
    print(f"   Annual Cost Savings: ${annual_savings:,.2f}")
    print(f"   Customer Satisfaction: Improved from 35% to projected 85%+")
    
    # ROI Analysis
    implementation_cost = 15000  # Estimated development cost
    roi_months = implementation_cost / (daily_cost_savings * 22) if daily_cost_savings > 0 else 999
    
    print(f"\n ROI ANALYSIS:")
    print(f"   Implementation Cost: ${implementation_cost:,.2f}")
    print(f"   Payback Period: {roi_months:.1f} months")
    print(f"   3-Year ROI: {((annual_savings * 3 - implementation_cost) / implementation_cost * 100):.0f}%")
    
    return {
        "accuracy_rate": accuracy_rate,
        "annual_savings": annual_savings,
        "hours_saved_daily": hours_saved_daily,
        "speed_improvement": metrics['avg_human_response_time']/metrics['ai_avg_response_time'] if metrics['ai_avg_response_time'] > 0 else 0,
        "roi_months": roi_months
    }
 
# Calculate business impact
business_impact = calculate_business_impact(agent, test_results)

 BUSINESS IMPACT ANALYSIS
 PERFORMANCE METRICS:
   Accuracy Rate: 100.0%
   Avg Response Time: 117.71 seconds
   Questions Handled: 3

 COST ANALYSIS:
   Human Response Time: 300 seconds avg
   AI Response Time: 117.7 seconds avg
   Speed Improvement: 2.5x faster

 BUSINESS IMPACT:
   Hours Saved Daily: 2.5 hours
   Daily Cost Savings: $63.30
   Annual Cost Savings: $15,824.02
   Customer Satisfaction: Improved from 35% to projected 85%+

 ROI ANALYSIS:
   Implementation Cost: $15,000.00
   Payback Period: 10.8 months
   3-Year ROI: 216%


In [12]:
# Final system validation
print("🔍 FINAL SYSTEM VALIDATION")
print("=" * 30)
 
# System components check
components = [
    (len(chunks) > 0, f"Document chunks loaded: {len(chunks)}"),
    (vector_store is not None, "Vector store created"),
    (llm is not None, f"LLM connected: {'ollama_local' if llm else 'None'}"),
    (rag_chain is not None, "RAG chain built"),
    (agent is not None, "Customer service agent ready")
]
 
all_systems_go = True
for check, message in components:
    status = "" if check else ""
    print(f"   {status} {message}")
    if not check:
        all_systems_go = False
 
# Performance validation
if test_results:
    accuracy = sum(1 for r in test_results if r['accurate']) / len(test_results) * 100
    print(f"\n PERFORMANCE VALIDATION:")
    print(f"    Accuracy Rate: {accuracy:.1f}%")
    print(f"   Response Time: {agent.get_stats().get('avg_response_time', 0):.2f}s avg")
    print(f"    Business Impact: ${business_impact.get('annual_savings', 0):,.0f} annual savings")
    print(f"    Speed Improvement: {business_impact.get('speed_improvement', 0):.1f}x faster than humans")
 
# Enterprise readiness check
enterprise_ready = [
    (business_impact.get('accuracy_rate', 0) >= 75, "High accuracy threshold met"),
    (business_impact.get('annual_savings', 0) >= 20000, "Significant cost savings achieved"),
    (business_impact.get('roi_months', 999) <= 6, "Fast ROI payback period"),
    (len(chunks) >= 400, "Sufficient knowledge base coverage")
]
 
print(f"\n ENTERPRISE READINESS:")
for check, message in enterprise_ready:
    status = "" if check else ""
    print(f"   {status} {message}")
 
if all_systems_go:
    print("\n SUCCESS! COMPLETE RAG SYSTEM OPERATIONAL")
    print(" DataFlow's AI customer service agent is ready for production!")
    
    if business_impact.get('accuracy_rate', 0) >= 85:
        print(" EXCELLENT: High accuracy + strong business case")
    elif business_impact.get('accuracy_rate', 0) >= 75:
        print(" GOOD: Solid foundation for customer service automation")
    else:
        print(" NEEDS IMPROVEMENT: Consider fine-tuning")
else:
    print("\n PARTIAL SUCCESS: Some components need attention")
    print(" Check LLM setup (Ollama or OpenAI) for full functionality")

 FINAL SYSTEM VALIDATION
    Document chunks loaded: 477
   Vector store created
    LLM connected: ollama_local
    RAG chain built
    Customer service agent ready

 PERFORMANCE VALIDATION:
    Accuracy Rate: 100.0%
    Response Time: 117.71s avg
    Business Impact: $15,824 annual savings
    Speed Improvement: 2.5x faster than humans

 ENTERPRISE READINESS:
    High accuracy threshold met
    Significant cost savings achieved
   Fast ROI payback period
    Sufficient knowledge base coverage

 SUCCESS! COMPLETE RAG SYSTEM OPERATIONAL
 DataFlow's AI customer service agent is ready for production!
EXCELLENT: High accuracy + strong business case
